In [21]:
#####ALWAYS RUN THIS CELL ONCE 
import os
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import json
import csv
import pandas as pd
import pandas.io.formats.excel
import re
from datetime import datetime
import io
import string
import re

# Keywords preparation

In [49]:
###Read the keyword file and create list

## read the keyword file
## file should be an xlsx file with the following structure (2 columns)
## Component	Keywords
xl = pd.ExcelFile("keywords_gender_restauration.xlsx") ## change path to the file here
df = xl.parse("Sheet1") ## name of the sheet where the keyword are - change if needed
df.fillna(method='ffill', inplace=True) 

## build a dict of keyword - group
group = df.set_index('Keywords').T.to_dict('list')

## lowercase everything and clean
group = dict((k.lower().strip(), list(map(lambda x:x.lower().strip(),v))) for k,v in group.items())

#print(group)


# Analysis

### Occurences

In [26]:
##### Folder to be used for the analysis -- contains all the text files
texts = './texts/'

In [31]:
###change the name of the output file here
with open("occurences.csv", "w" , encoding="utf-8-sig") as key_file:
    #headers
    ### file names
    key_file.write("Keywords")
    key_file.write(";")
    ### keywords
    for k in group:
        key_file.write(k)
        key_file.write(";")
    key_file.write("\n")   
    ### add group information
    key_file.write("Group")
    key_file.write(";")        
    for keyword in group:
        key_file.write(group[keyword][0])
        key_file.write(";")
    key_file.write("\n")
    
    for filename in os.listdir(texts):
        if(("DS_Store") in filename): ## mac issue
            continue
        with open(texts+filename, encoding="ISO-8859-1") as text_file:
            data = text_file.read().replace('\n', ' ') ## remove end of lines
            data = data.lower()
            data = data.replace("- ", "") ## end of line
            
            key_file.write(filename)
            key_file.write(";")
            
            for keyword in group:
                if ('*' in keyword):
                    occ = 0
                    words = keyword.split('*')
                     
                    ## works when one *
                    if(len(words)==2):
                        x = re.search(words[0]+"[^\s]+"+words[1], data)
                        if(x):
                            occ+=1
                    elif(len(words)==3):
                        x = re.search(words[0]+"[^\s]+"+words[1]+"[^\s]+"+words[2], data)
                        if(x):
                            occ+=1
                    else:
                        print(keyword)

                    key_file.write(str(occ))
                    key_file.write(";")

                else:
                    occ = data.count(keyword.lower())
                    #print(keyword +" " + str(occ))
                    key_file.write(str(occ))
                    key_file.write(";")
            key_file.write("\n")

### Sentences extraction

In [32]:
###change the name of the output file here
with io.open("sentences.csv", "w" , encoding="utf-8") as key_file:
    #headers
    key_file.write("Sentence")
    key_file.write(";")
    key_file.write("File Name")
    key_file.write(";")
    key_file.write("Keyword")
    key_file.write(";")
    key_file.write("Group")
    key_file.write("\n")
    
    for filename in os.listdir(texts):
        if(("DS_Store") in filename):
            continue
        with io.open(texts+filename, 'r', encoding="ISO-8859-1") as text_file:
            
            data = text_file.read()
            sentences = sent_tokenize(data)
            
            sentences_clean = []
            for sentence in sentences:
                ## filter non ascii non printable characters
                sentence = "".join(c for c in sentence if c.isprintable() or c in "\n\t")
                sentence = sentence.replace('\n', ' ') ## remove end of line
                sentence = sentence.replace('\\n', ' ') ## remove end of lines
                sentence = sentence.lower()
                sentence = sentence.replace("- ", "")
                sentence = re.sub(' {*}', ' ', sentence)
                
                sentences_clean.append(sentence)
        
        
        sentenceBeforeAfter = []
        
        for a, b, c in zip(sentences_clean, sentences_clean[1:], sentences_clean[2:]):          
            for k in group: ### change the keywords list here
                match = None
                if ('*' in k):
                    words = k.split('*')
                    
                    if(len(words)==2):
                        x = re.search(words[0]+"[^\s]+"+words[1], b)
                        if(x):
                            match = True
                    elif(len(words)==3):
                        x = re.search(words[0]+"[^\s]+"+words[1]+"[^\s]+"+words[2], b)
                        if(x):
                            match = True
                    else:
                        print(keyword)
                else:
                    if(k.lower() in b):
                        match = True

                if match:  
                    key_file.write("\"")
                    key_file.write(a.replace("\"", ""))
                    key_file.write(b.replace("\"", ""))
                    key_file.write(c.replace("\"", ""))
                    key_file.write("\"")
                    key_file.write(";")
                    key_file.write("\"")
                    key_file.write(filename)
                    key_file.write("\"")
                    key_file.write(";")
                    key_file.write("\"")
                    key_file.write(k)
                    key_file.write("\"")
                    key_file.write(";")
                    key_file.write("\"")
                    key_file.write(group[k][0])
                    key_file.write("\"")
                    key_file.write("\n")

# Final file generation

In [46]:
##read the output files
occ1 = pd.read_csv('occurences.csv', sep=";")
occ1.dropna(how='all', axis='columns', inplace=True)
occ1 = occ1.T
sen1 = pd.read_csv('sentences.csv', sep=";", index_col=0)

### build the excel file
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('results.xlsx', engine='xlsxwriter',  options={'strings_to_numbers': True})
workbook = writer.book
##
pd.io.formats.excel.header_style = None

# Write each dataframe to a different worksheet.
occ1.to_excel(writer, sheet_name='Occurences',header=False)
sen1.to_excel(writer, sheet_name='Sentences')

###formatting updated from http://pbpython.com/improve-pandas-excel-output.html
number_rows = len(occ1.index)
# Define our range for the color formatting
color_range = "C2:ALF{}".format(number_rows+1)
# Add a format. Light red fill with dark red text.
format1 = workbook.add_format({'bg_color': '#FFC7CE',
                               'font_color': '#9C0006'})

worksheet = writer.sheets['Occurences']

# Highlight the values > 0 in Red
worksheet.conditional_format(color_range, {'type':     'cell',
                                            'criteria': '>',
                                            'value':    0,
                                            'format': format1})
# Close the Pandas Excel writer and output the Excel file.
writer.save()